<a href="https://colab.research.google.com/github/Prem20560404/CSproject/blob/master/custom.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd /content/drive/MyDrive/csproject

/content/drive/MyDrive/csproject


In [ ]:
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y


Found existing installation: Keras 2.0.8
Uninstalling Keras-2.0.8:
  Successfully uninstalled Keras-2.0.8
Found existing installation: Keras-Preprocessing 1.1.2
Uninstalling Keras-Preprocessing-1.1.2:
  Successfully uninstalled Keras-Preprocessing-1.1.2
Found existing installation: keras-vis 0.4.1
Uninstalling keras-vis-0.4.1:
  Successfully uninstalled keras-vis-0.4.1
Found existing installation: tensorflow 1.13.1
Uninstalling tensorflow-1.13.1:
  Successfully uninstalled tensorflow-1.13.1
Found existing installation: h5py 2.10.0
Uninstalling h5py-2.10.0:
  Successfully uninstalled h5py-2.10.0


In [ ]:
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0
!pip install -U scikit-image==0.16.2

  Using cached tensorflow-1.13.1-cp37-cp37m-manylinux1_x86_64.whl (92.6 MB)
     |████████████████████████████████| 42 kB 1.1 MB/s 
     |████████████████████████████████| 4.1 MB 29.5 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
kapre 0.3.6 requires tensorflow>=2.0.0, but you have tensorflow 1.13.1 which is incompatible.
  Using cached Keras-2.0.8-py2.py3-none-any.whl (276 kB)
  Using cached h5py-2.10.0-cp37-cp37m-manylinux1_x86_64.whl (2.9 MB)
  Attempting uninstall: h5py
    Found existing installation: h5py 3.6.0
    Uninstalling h5py-3.6.0:
      Successfully uninstalled h5py-3.6.0


In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
from mrcnn.visualize import display_instances
import matplotlib.pyplot as plt

ModuleNotFoundError: ignored

In [ ]:
ROOT_DIR = "/content/drive/MyDrive/csproject"

In [ ]:
sys.path.append(ROOT_DIR)
from mrcnn.config import Config
from mrcnn import model as modellib, utils


COCO_WEIGHTS_PATH = os.path.join(ROOT_DIR, "mask_rcnn_coco.h5")

DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

Using TensorFlow backend.


In [ ]:
class CustomConfig(Config):

    NAME = "object"


    IMAGES_PER_GPU = 2


    NUM_CLASSES = 1 + 3


    STEPS_PER_EPOCH = 5

    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "car")
        self.add_class("object", 2, "truck")
        self.add_class("object",3,"ocean")


        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir, subset)

        
        annotations1 = json.load(open("/content/drive/MyDrive/csproject/Dataset/train/train_json.json"))
        print(annotations1)
        annotations = list(annotations1.values())


        annotations = [a for a in annotations if a['regions']]

        for a in annotations:
            polygons = [r['shape_attributes'] for r in a['regions']]
            print([s['region_attributes'] for s in a['regions'] if 'names' not in s['region_attributes']])
            objects = [s['region_attributes']['names'] for s in a['regions']if 'names' in s['region_attributes']]
            print("objects:",objects)

            name_dict = {"car": 1,"truck": 2, "ocean": 3}
                                                                                                              
            num_ids = [name_dict[a] for a in objects]
     

            print("numids",num_ids)
            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image("object", image_id=a['filename'],path=image_path, width=width, height=height, polygons=polygons, num_ids=num_ids)
            

    def load_mask(self, image_id):

        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)


        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])], dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):

        	rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])

        	mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids


    def image_reference(self, image_id):
        """Return the path of the image."""
        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)


    

In [ ]:
def train(model):
      dataset_train = CustomDataset()
      dataset_train.load_custom("/content/drive/MyDrive/csproject/Dataset", "train")
      dataset_train.prepare()

  
      dataset_val = CustomDataset()
      dataset_val.load_custom("/content/drive/MyDrive/csproject/Dataset", "val")
      dataset_val.prepare()

    
      print("Training network heads")
      model.train(dataset_train, dataset_val, learning_rate=config.LEARNING_RATE, epochs=5, layers='heads')

In [ ]:
config = CustomConfig()
model = modellib.MaskRCNN(mode="training", config=config, model_dir=DEFAULT_LOGS_DIR)
weights_path = COCO_WEIGHTS_PATH

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [ ]:
if not os.path.exists(weights_path):
  utils.download_trained_weights(weights_path)

model.load_weights(weights_path, by_name=True, exclude=[
            "mrcnn_class_logits", "mrcnn_bbox_fc",
            "mrcnn_bbox", "mrcnn_mask"])

In [ ]:
train(model)

{'1.jpg171916': {'filename': '1.jpg', 'size': 171916, 'regions': [{'shape_attributes': {'name': 'polygon', 'all_points_x': [66, 72, 92, 95, 88, 72, 67], 'all_points_y': [234, 241, 245, 227, 220, 216, 218]}, 'region_attributes': {'names': 'car'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [6, 31, 38, 36, 11, 4], 'all_points_y': [325, 329, 336, 355, 352, 346]}, 'region_attributes': {'names': 'car'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [348, 369, 373, 370, 363, 347, 342], 'all_points_y': [237, 233, 224, 218, 216, 218, 226]}, 'region_attributes': {'names': 'car'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [387, 403, 409, 409, 404, 386, 381], 'all_points_y': [229, 224, 218, 209, 204, 210, 216]}, 'region_attributes': {'names': 'car'}}, {'shape_attributes': {'name': 'polygon', 'all_points_x': [404, 426, 426, 424, 416, 398], 'all_points_y': [304, 298, 285, 278, 277, 286]}, 'region_attributes': {'names': 'car'}}, {'shape_attributes': {'name':

/usr/local/lib/python3.7/dist-packages/tensorflow/python/ops/gradients_impl.py:110: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1987: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


Epoch 1/5
4/5 [=======================>......] - ETA: 154s - loss: 5.0898 - rpn_class_loss: 0.2909 - rpn_bbox_loss: 0.4870 - mrcnn_class_loss: 2.3209 - mrcnn_bbox_loss: 0.9300 - mrcnn_mask_loss: 1.0610

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:2142: UserWarning: Using a generator with `use_multiprocessing=True` and multiple workers may duplicate your data. Please consider using the`keras.utils.Sequence class.
  UserWarning('Using a generator with `use_multiprocessing=True`'


5/5 [==============================] - 3707s - loss: 4.6455 - rpn_class_loss: 0.2671 - rpn_bbox_loss: 0.5095 - mrcnn_class_loss: 1.9144 - mrcnn_bbox_loss: 0.9193 - mrcnn_mask_loss: 1.0352 - val_loss: 2.8918 - val_rpn_class_loss: 0.2108 - val_rpn_bbox_loss: 0.4428 - val_mrcnn_class_loss: 0.5845 - val_mrcnn_bbox_loss: 0.6664 - val_mrcnn_mask_loss: 0.9872
Epoch 2/5
5/5 [==============================] - 3552s - loss: 2.5570 - rpn_class_loss: 0.1747 - rpn_bbox_loss: 0.4326 - mrcnn_class_loss: 0.4526 - mrcnn_bbox_loss: 0.5909 - mrcnn_mask_loss: 0.9062 - val_loss: 2.0092 - val_rpn_class_loss: 0.0956 - val_rpn_bbox_loss: 0.3083 - val_mrcnn_class_loss: 0.3158 - val_mrcnn_bbox_loss: 0.4884 - val_mrcnn_mask_loss: 0.8011
Epoch 3/5
5/5 [==============================] - 3484s - loss: 1.6315 - rpn_class_loss: 0.0612 - rpn_bbox_loss: 0.2491 - mrcnn_class_loss: 0.2072 - mrcnn_bbox_loss: 0.3868 - mrcnn_mask_loss: 0.7272 - val_loss: 1.2842 - val_rpn_class_loss: 0.0407 - val_rpn_bbox_loss: 0.1685 - val_